# Perform data preprocessing with pycytominer on single cell features

Note: Single cell is represented by only nuclei compartment features which was used to extract features across all three channels.

## Import libraries

In [1]:
import pathlib
import pprint

import pandas as pd
from pycytominer import annotate, normalize, feature_select

## Set paths and variables

In [2]:
# Path to dir with converted profiles per plate (each plate as a folder)
cleaned_dir = pathlib.Path("./data/cleaned_sc_profiles")

# path for plate map directory
platemap_dir = pathlib.Path(f"../0.download_data/metadata/platemaps")

# Output dir for the files to be saved to
output_dir = pathlib.Path("./data/single_cell_profiles")
output_dir.mkdir(exist_ok=True, parents=True)

# Extract the plate names from the file names
plate_names = [file.stem.split("_")[0] for file in platemap_dir.glob("*_platemap.csv")]

# operations to perform for feature selection
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
slide2
slide1
slide4
slide3


## Create dictionary with unique paths for each plate

In [3]:
# create plate info dictionary
plate_info_dictionary = {
    name: {
        "profile_path": str(
            pathlib.Path(f"{cleaned_dir}/{name}_sc_cleaned.parquet").resolve(strict=True)
        ),
        "platemap_path": str(
            pathlib.Path(f"{platemap_dir}/{name}_platemap.csv").resolve(strict=True)
        ),
    }
    for name in plate_names
}

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'slide1': {   'platemap_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/0.download_data/metadata/platemaps/slide1_platemap.csv',
                  'profile_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/4.preprocess_features/data/cleaned_sc_profiles/slide1_sc_cleaned.parquet'},
    'slide2': {   'platemap_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/0.download_data/metadata/platemaps/slide2_platemap.csv',
                  'profile_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/4.preprocess_features/data/cleaned_sc_profiles/slide2_sc_cleaned.parquet'},
    'slide3': {   'platemap_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/0.download_data/metadata/platemaps/slide3_platemap.csv',
                  'profile_path': '/media/18tbdrive/Github_Repositories/nuclear_speckle_image_analysis/4.preprocess_features/data/cleaned_sc_profiles/slide3_sc

## Perform preprocessing on single cell features

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Performing pycytominer pipeline for {plate}")
    # Set output paths per preprocessing step
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_feature_selected.parquet")
    )

    # Load in the converted profile to be used in the first step
    profile_df = pd.read_parquet(info["profile_path"])

    # Load in platemap file with most relevant columns for annotation
    platemap_df = pd.read_csv(info["platemap_path"], usecols=["Well", "CellLine", "Condition"])

    # Step 1: Annotation
    annotate(
        profiles=profile_df,
        platemap=platemap_df,
        join_on=["Metadata_Well", "Image_Metadata_Well"],
        output_file=output_annotated_file,
        output_type="parquet",
    )

    # Load the annotated parquet file to fix metadata columns names
    annotated_df = pd.read_parquet(output_annotated_file)

    # Rename columns using the rename() function
    column_name_mapping = {
        "Image_Metadata_Site": "Metadata_Site",
        "Image_Count_Nuclei": "Metadata_Nuclei_Site_Count",
        "Nuclei_Location_Center_X": "Metadata_Nuclei_Location_Center_X",
        "Nuclei_Location_Center_Y": "Metadata_Nuclei_Location_Center_Y",
    }

    annotated_df.rename(columns=column_name_mapping, inplace=True)

    # Save the modified DataFrame back to the same location
    annotated_df.to_parquet(output_annotated_file, index=False)

    # Step 2: Normalization
    normalized_df = normalize(
        profiles=output_annotated_file,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    # Step 3: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        output_file=output_feature_select_file,
        output_type="parquet",
    )
    print(
        f"Annotation, normalization, and feature selection have been performed for {plate}"
    )

Performing pycytominer pipeline for slide2
Annotation, normalization, and feature selection have been performed for slide2
Performing pycytominer pipeline for slide1
Annotation, normalization, and feature selection have been performed for slide1
Performing pycytominer pipeline for slide4
Annotation, normalization, and feature selection have been performed for slide4
Performing pycytominer pipeline for slide3
Annotation, normalization, and feature selection have been performed for slide3


## Check example output file to confirm that the process worked

In [5]:
# Check output file
test_df = pd.read_parquet("./data/single_cell_profiles/slide1_sc_annotated.parquet")

print(test_df.shape)
test_df.head(2000)

(69631, 587)


,Metadata_CellLine,Metadata_Condition,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Nuclei_Site_Count,Image_FileName_DAPI,Nuclei_AreaShape_Area,Nuclei_AreaShape_BoundingBoxArea,...,Nuclei_Texture_Variance_A647_3_02_256,Nuclei_Texture_Variance_A647_3_03_256,Nuclei_Texture_Variance_DAPI_3_00_256,Nuclei_Texture_Variance_DAPI_3_01_256,Nuclei_Texture_Variance_DAPI_3_02_256,Nuclei_Texture_Variance_DAPI_3_03_256,Nuclei_Texture_Variance_GOLD_3_00_256,Nuclei_Texture_Variance_GOLD_3_01_256,Nuclei_Texture_Variance_GOLD_3_02_256,Nuclei_Texture_Variance_GOLD_3_03_256
0,786O,NTC,1,slide1,A1,M10,25,slide1_A1_M10_CH0_Z09_illumcorrect.tiff,1483.0,2550.0,...,14.731878,13.950219,2.508336,2.486800,2.539577,2.484139,2.586139,2.685200,2.649136,2.497674
1,786O,NTC,1,slide1,A1,M10,25,slide1_A1_M10_CH0_Z09_illumcorrect.tiff,1378.0,1974.0,...,50.715492,50.584647,11.958722,11.757222,11.748338,12.026326,8.242161,8.072166,8.239189,8.313167
2,786O,NTC,1,slide1,A1,M10,25,slide1_A1_M10_CH0_Z09_illumcorrect.tiff,1345.0,1974.0,...,121.489707,122.096126,50.735363,51.805624,50.608420,51.418746,28.538125,29.204032,28.407457,28.739371
3,786O,NTC,1,slide1,A1,M10,25,slide1_A1_M10_CH0_Z09_illumcorrect.tiff,1403.0,5130.0,...,0.000000,0.000000,47.224646,50.686761,46.432874,47.049561,0.000000,0.000000,0.000000,0.000000
4,786O,NTC,1,slide1,A1,M10,25,slide1_A1_M10_CH0_Z09_illumcorrect.tiff,1157.0,1950.0,...,197.081725,197.985838,116.485446,121.235388,115.894563,117.135912,40.604219,41.407578,40.214090,41.106731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,786O,ALY kd5,164,slide1,B1,M34,45,slide1_B1_M34_CH0_Z09_illumcorrect.tiff,1899.0,2478.0,...,165.830160,166.073994,191.702396,191.443504,199.506105,194.313025,37.009665,36.766357,37.566023,37.290265
1996,786O,ALY kd5,164,slide1,B1,M34,45,slide1_B1_M34_CH0_Z09_illumcorrect.tiff,1468.0,3087.0,...,58.983392,60.255646,6.953567,7.003456,7.063025,6.898384,15.800336,15.654464,15.549256,15.801234
1997,786O,ALY kd5,164,slide1,B1,M34,45,slide1_B1_M34_CH0_Z09_illumcorrect.tiff,1271.0,1813.0,...,186.951117,187.410041,147.986467,138.438177,142.843115,145.038674,45.787944,46.933356,45.560333,45.752716
1998,786O,ALY kd5,164,slide1,B1,M34,45,slide1_B1_M34_CH0_Z09_illumcorrect.tiff,909.0,1395.0,...,541.260235,558.393558,276.367134,282.484421,279.763835,283.239187,241.145598,236.920202,224.197230,235.216142
